# Imports and Constants

In [25]:
import sys
from collections import deque

# Define Process

In [39]:
class Process:
    def __init__(self, id, arrive_time, burst_time):
        self.id = id
        self.arrive_time = arrive_time
        self.burst_time = burst_time
        self.wait_start_time = arrive_time
        self.processed_time = 0
        
    #for printing purpose
    def __repr__(self):
        return ('[id %d : arrive_time %d,  burst_time %d]'%(self.id, self.arrive_time, self.burst_time))

# Define Schedulers
* Input: process_list, [time_quantum (Positive Integer), alpha]
* Output_1 : Schedule list contains pairs of (time_stamp, proccess_id) indicating the time switching to that proccess_id
* Output_2 : Average Waiting Time


## First Come First Serve

In [12]:
def FCFS_scheduling(process_list):
    #store the (switching time, proccess_id) pair
    schedule = []
    current_time = 0
    waiting_time = 0
    for process in process_list:
        if(current_time < process.arrive_time):
            current_time = process.arrive_time
            
        schedule.append((current_time, process.id))
        waiting_time = waiting_time + (current_time - process.arrive_time)
        current_time = current_time + process.burst_time
    average_waiting_time = waiting_time / float(len(process_list))
    
    return schedule, average_waiting_time

In [48]:
process_list = [
    Process(0, 0, 10),
    Process(1, 5, 10),
    Process(2, 25, 10)
]

schedule, avg = FCFS_scheduling(process_list)
print schedule
print avg

[(0, 0), (10, 1), (25, 2)]
1.66666666667


## Round Robin
Scheduling process_list on round robin policy with time_quantum

In [50]:
def RR_scheduling(process_list, time_quantum ):
    n = float(len(process_list))
    
    schedule = []
    current_time = 0
    waiting_time = 0
    
    current_process = None
    queue = deque()
    
    while len(queue) > 0 or len(process_list) > 0:
        # Handle time-gaps
        if len(queue) <= 0:
            current_process = process_list[0]
            process_list = process_list[1:]
            current_time = current_process.arrive_time
            schedule.append((current_time, current_process.id))
        # RR process queue
        else:
            prev_process = current_process
            current_process = queue.popleft()
            if current_process.id != prev_process.id:
                prev_process.wait_start_time = current_time
                waiting_time = waiting_time + current_time - current_process.wait_start_time
                schedule.append((current_time, current_process.id))
        
        # If process finish early reclaim time, otherwise preempt at quanta
        process_time = min(
            time_quantum, 
            current_process.burst_time - current_process.processed_time)
        current_time = current_time + process_time
        
        # Process arrivals during quanta
        while len(process_list) > 0 and process_list[0].arrive_time <= current_time:
            queue.append(process_list[0])
            process_list = process_list[1:]
        
        # Re-enqueue current process if not finished
        current_process.processed_time = current_process.processed_time + process_time
        if current_process.processed_time < current_process.burst_time:
            queue.append(current_process)
    
    return (schedule, waiting_time / n)

In [51]:
process_list = [
    Process(0, 0, 10),
    Process(1, 5, 10),
    Process(2, 25, 10)
]

schedule, avg = RR_scheduling(process_list, 2)
print schedule
print avg

[(0, 0), (6, 1), (8, 0), (10, 1), (12, 0), (14, 1), (25, 2)]
3.0


## Shortest Remaining Time First 
Scheduling process_list on SRTF, using process.burst_time to calculate the remaining time of the current process

In [5]:
def SRTF_scheduling(process_list):
    return (["to be completed"], 0.0)

## Shortest Job First
Scheduling using future prediction SJF without using information from process.burst_time

Let initial guess = 5 time units.

In [6]:

def SJF_scheduling(process_list, alpha):
    return (["to be completed"],0.0)

# Read/Write Input/Output

In [7]:
def read_input(file_name):
    result = []
    with open(file_name) as f:
        for line in f:
            array = line.split()
            if (len(array)!= 3):
                print ("wrong input format")
                exit()
            result.append(Process(int(array[0]),int(array[1]),int(array[2])))
    return result

In [8]:
def write_output(file_name, schedule, avg_waiting_time):
    with open(file_name,'w') as f:
        for item in schedule:
            f.write(str(item) + '\n')
        f.write('average waiting time %.2f \n'%(avg_waiting_time))


# Main Call

In [9]:
process_list = read_input('input.txt')
print ("printing input ----")
for process in process_list:
    print (process)

print ("simulating FCFS ----")
FCFS_schedule, FCFS_avg_waiting_time =  FCFS_scheduling(process_list)
write_output('FCFS.txt', FCFS_schedule, FCFS_avg_waiting_time )

print ("simulating RR ----")
RR_schedule, RR_avg_waiting_time =  RR_scheduling(process_list,time_quantum = 2)
write_output('RR.txt', RR_schedule, RR_avg_waiting_time )

print ("simulating SRTF ----")
SRTF_schedule, SRTF_avg_waiting_time =  SRTF_scheduling(process_list)
write_output('SRTF.txt', SRTF_schedule, SRTF_avg_waiting_time )

print ("simulating SJF ----")
SJF_schedule, SJF_avg_waiting_time =  SJF_scheduling(process_list, alpha = 0.5)
write_output('SJF.txt', SJF_schedule, SJF_avg_waiting_time )

printing input ----
[id 0 : arrive_time 0,  burst_time 9]
[id 1 : arrive_time 1,  burst_time 8]
[id 2 : arrive_time 2,  burst_time 2]
[id 3 : arrive_time 5,  burst_time 2]
[id 3 : arrive_time 30,  burst_time 5]
[id 1 : arrive_time 31,  burst_time 2]
[id 2 : arrive_time 32,  burst_time 6]
[id 0 : arrive_time 38,  burst_time 8]
[id 2 : arrive_time 60,  burst_time 7]
[id 0 : arrive_time 62,  burst_time 2]
[id 1 : arrive_time 65,  burst_time 3]
[id 3 : arrive_time 66,  burst_time 8]
[id 1 : arrive_time 90,  burst_time 10]
[id 0 : arrive_time 95,  burst_time 10]
[id 2 : arrive_time 98,  burst_time 9]
[id 3 : arrive_time 99,  burst_time 8]
simulating FCFS ----
simulating RR ----
simulating SRTF ----
simulating SJF ----
